In [49]:
from functools import partial
from sqlite3 import connect

import astroplan as ap
from astropy.coordinates import EarthLocation
from astropy.table import Table
from astropy.time import Time
import astropy.units as u
from icecream import ic, install
import numpy as np
import pandas as pd

import astropaul.targetlistcreator as tlc
import astropaul.lbt as lbt
import astropaul.html as html
import astropaul.phase as ph
import astropaul.priority as pr

install() # icecream will now work in imported packages

%load_ext autoreload
%autoreload 2

# from itables import init_notebook_mode

# init_notebook_mode(
#     all_interactive=True,
#     connected=True,
# )

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [68]:
conn = connect("../../django/TargetDB/db.sqlite3")
html_dir = "html"

session = tlc.ObservingSession(ap.Observer.at_site("APO"))
session.add_full_day("2025-03-15")
session.add_full_day("2025-03-16")
session.add_half_day("2025-03-18", first_half=False)
session.add_full_day("2025-03-19")

phase_event_defs = [
    ph.PhaseEventDef("Not in Eclipse", partial(ph.calc_time_of_gress, ingress=False)),
    ph.PhaseEventDef("Eclipse", partial(ph.calc_time_of_gress, ingress=True)),
]

name = "APO Observing List 2025-03-15"
creator = tlc.TargetListCreator(name=name, connection=conn)
creator.steps = [
    tlc.add_targets,
    tlc.add_tess,
    tlc.add_coords,
    partial(tlc.hide_cols, prefix="TESS "),
    tlc.add_speckle,
    tlc.add_pepsi,
    tlc.add_lists,
    # partial(tlc.filter_targets, criteria=lambda df: (df["Num Speckle"] > 0)),
    # partial(tlc.filter_targets, criteria=lambda df: (df["List Speckle Resolved"])),
    partial(tlc.add_observability, observing_session=session, calc_max_altitude=True, calc_moon_distance=True),
    partial(tlc.filter_targets, criteria=lambda df: (df["Observable Every Night"])),
    partial(tlc.filter_targets, criteria=lambda df: (df["Observable Max Alt"] > 40)),
    tlc.add_ephemerides,
    partial(tlc.add_phase_events, observing_session=session, phase_event_defs=phase_event_defs),
]
tl = creator.calculate(verbose=False)

print(tl.summarize())
# tl.target_list

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\erfa\core.py:4620: RuntimeWarning: invalid value encountered in ld
  p1 = ufunc.ld(bm, p, q, e, em, dlim)
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\erfa\core.py:19004: RuntimeWarning: invalid value encountered in anp
  c_retval = ufunc.anp(a)
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\erfa\core.py:4036: RuntimeWarning: invalid value encountered in atioq
  aob, zob, hob, dob, rob = ufunc.atioq(ri, di, astrom)
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\erfa\core.py:4620: RuntimeWarning: invalid value encountered in ld
  p1 = ufunc.ld(bm, p, q, e, em, dlim)
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\erfa\core.py:19004: RuntimeWarning: invalid value encountered in anp
  c_retval = ufunc.anp(a)
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\erfa\core.py:4036: RuntimeWarning: inval

APO Observing List 2025-03-15
31 targets:
      31 QuadEB
Column Count (primary, secondary):
    Target: (1, 2)
    Coordinates: (6, 3)
    Count: (2, 0)
    List : (0, 10)
    Observable: (4, 12)
Associated tables:
     149 rows,  6 columns: Speckle
     103 rows,  9 columns: PEPSI
       4 rows,  2 columns: Lunar Phases
     102 rows,  6 columns: Ephem
     546 rows,  8 columns: Phase Events



In [69]:
altitude_categories = [
    ((-90, 30), 0),
    ((30, 45), 0.85),
    ((45, 60), 0.95),
    ((60, 90), 1),
]

science_tl = tl.copy()
science_tl.target_list = science_tl.target_list[science_tl.target_list["Target Type"] == "QuadEB"]
pl = pr.PriorityList(science_tl, session, interval=60 * u.min)
pr.calculate_altitude_priority(pl, altitude_categories=altitude_categories)
pr.calculate_eclipse_priority(pl, no_eclipse_score=0, min_altitude=30 * u.deg)
pr.calculate_overall_priority(pl)
pr.aggregate_target_priorities(pl, skip_column_threshold=0)
pl.categorize_priorities(bins=[0.00, 0.20, 0.40, 0.6, 1.00], labels=["", "*", "* *", "* * *"])

html.render_observing_pages(tl, pl, {}, "html")
pl.categorical_priorities[0]

,TIC 443862276,TIC 27543409,TIC 779824,TIC 95928255,TIC 414969157,TIC 219006972,TIC 161043618,TIC 20212631,TIC 219469945,TIC 356318101,...,TIC 322727163,TIC 375325607,TIC 273919067,TIC 358422952,TIC 344541836,TIC 427092089,TIC 240256832,TIC 160514618,TIC 278465736,TIC 327885074
2025-03-15 02:00:00,,,,,,,,,,,...,,,,,,,,,,
2025-03-15 03:00:00,,,,,,,,,,,...,,,,,,,,,,
2025-03-15 04:00:00,,* * *,,,,,,,,,...,,,,,,,,,,
2025-03-15 05:00:00,,* * *,,,,,,,,,...,,,,,,,,,,
2025-03-15 06:00:00,,,,,,,* * *,* * *,,,...,,,,,,,,,,
2025-03-15 07:00:00,,,,,,,* * *,* * *,,,...,,,,,,,,,,
2025-03-15 08:00:00,,,,,,,* * *,* * *,,,...,,,,,,,,,,
2025-03-15 09:00:00,,,,,,,* * *,,,,...,,,,,,,,,,
2025-03-15 10:00:00,,,,,,,* * *,,,,...,,,,,,,,,,
2025-03-15 11:00:00,,,,,,,* * *,,,,...,,,,,,,,,,
